In [1]:
# License: BSD
# Author: Sasank Chilamkurthy

from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from random import randrange
import torch.nn.functional as F


plt.ion()   # interactive mode

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split


dat_race = pd.read_csv('dnd_race.csv')
dat_class = pd.read_csv('dnd_class.csv')

dat_race['image_path'] = dat_race['label']+'/'+dat_race['image_number']
dat_class['image_path'] = dat_class['label']+'/'+dat_class['image_number']

dat_race = dat_race.rename(index=str, columns={"label": "race"})
dat_class = dat_class.rename(index=str, columns={"label": "class"})
dat_class.head(1)


X_race = dat_race['image_path']
X_class = dat_class['image_path']

y_race = pd.get_dummies(dat_race['race'])
y_class = pd.get_dummies(dat_class['class'])


In [3]:
dat_class['class'].value_counts()

rogue      148
paladin    147
wizard     119
Name: class, dtype: int64

In [4]:
148/dat_class.shape[0]

0.357487922705314

In [5]:
dat_race.race.value_counts()

elf           188
gnome         122
dragonborn    120
Name: race, dtype: int64

In [6]:
188/dat_race.shape[0]

0.4372093023255814

In [7]:
X_race_train, X_race_test, y_race_train, y_race_test = train_test_split(X_race, y_race, test_size=0.20, random_state=42)
X_class_train, X_class_test, y_class_train, y_class_test = train_test_split(X_class, y_class, test_size=0.20, random_state=42)


In [8]:
X_race_train = X_race_train.values.tolist()
X_race_test = X_race_test.values.tolist()


X_class_train = X_class_train.values.tolist()
X_class_test = X_class_test.values.tolist()

race_shape = y_race_train.shape[1]
class_shape = y_class_train.shape[1]


y_race_train = y_race_train.values.tolist()
y_race_test = y_race_test.values.tolist()

y_class_train = y_class_train.values.tolist()
y_class_test = y_class_test.values.tolist()

In [9]:
len(y_race_train)

344

In [10]:
len(y_class_train)

331

In [11]:
#for i in dataset_style:
#    print(i,dat[i].sum(),dat[i].sum()/dat.shape[0])

In [12]:
#for i in dataset_rest:
#    print(i,dat[i].sum(),dat[i].sum()/dat.shape[0])

In [13]:
import random
flip = random.randint(0, 1)
flip

0

In [14]:
class fgo_dataset(Dataset):
    def __init__(self,king_of_lists, transform=None):
        """
        gender_img, style_img, gender_pred,style_pred 
        
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.king_of_lists = king_of_lists
        self.transform = transform
        
    def __getitem__(self,index):
        
        #random_index = randrange(len(self.king_of_lists[index]))
        
        img1 = Image.open('images/'+self.king_of_lists[0][index])
        img1 = img1.convert('RGB')
        
        #img2 = Image.open('images/'+self.king_of_lists[1][index])
        #img2 = img2.convert('RGB')
        
        
        gender = self.king_of_lists[1][index] # gender
        #color = self.king_of_lists[3][index] # region
        
        if self.transform is not None:
            img1 = self.transform(img1)

        list_of_labels = [torch.from_numpy(np.array(gender))]
       
        return img1, list_of_labels[0]
    
    def __len__(self):
        return len(self.king_of_lists[0])

In [15]:
batch_size = 8
# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((256,256)),
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((224,224)),
        #transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

train_lists = [X_race_train, y_race_train]
test_lists = [X_race_test, y_race_test ]

training_dataset = fgo_dataset(king_of_lists = train_lists,
                               transform = data_transforms['train'] )

test_dataset = fgo_dataset(king_of_lists = test_lists,
                           transform = data_transforms['val'] )

dataloaders_dict = {'train': torch.utils.data.DataLoader(training_dataset, batch_size=batch_size, shuffle=True, num_workers=0),
                   'val':torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
                   }
dataset_sizes = {'train':len(train_lists[0]),
                'val':len(test_lists[0])}


train_class_lists = [X_class_train, y_class_train]
test_class_lists = [X_class_test, y_class_test ]

training_style_dataset = fgo_dataset(king_of_lists = train_class_lists,
                               transform = data_transforms['train'] )

test_style_dataset = fgo_dataset(king_of_lists = test_class_lists,
                           transform = data_transforms['val'] )

dataloaders_style_dict = {'train': torch.utils.data.DataLoader(training_style_dataset, batch_size=batch_size,drop_last =True, shuffle=True, num_workers=0),
                   'val':torch.utils.data.DataLoader(test_style_dataset, batch_size=batch_size,drop_last =True, shuffle=True, num_workers=0)
                   }
dataloaders_style_sizes = {'train':len(train_lists[0]),
                'val':len(test_lists[0])}

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [16]:
print(len(dataloaders_style_dict['train']),len(dataloaders_style_dict['val']))

41 10


In [17]:
print(len(dataloaders_dict['train']),len(dataloaders_dict['val']))

43 11


In [18]:
model_ft = models.resnet50(pretrained=True)
#for param in model_ft.parameters():
#    param.requires_grad = False
print(model_ft)
#num_ftrs = model_ft.classifier[6].in_features
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, 512)

#race_shape = y_race_train.shape[1]
#class_shape = y_class_train.shape[1]
class multi_output_model(torch.nn.Module):
    def __init__(self, model_core):
        super(multi_output_model, self).__init__()
        
        self.resnet_model = model_core
        
        self.x1 =  nn.Linear(512,512)
        nn.init.xavier_normal_(self.x1.weight)
        self.x1a =  nn.Linear(512,512)
        nn.init.xavier_normal_(self.x1a.weight)
        
        self.x2 =  nn.Linear(512,256)
        nn.init.xavier_normal_(self.x2.weight)
        self.x2a =  nn.Linear(256,256)
        nn.init.xavier_normal_(self.x2a.weight)
        
        self.x3 =  nn.Linear(256,128)
        nn.init.xavier_normal_(self.x3.weight)
        self.x3a =  nn.Linear(128,128)
        nn.init.xavier_normal_(self.x3a.weight)
        #comp head 1
        
        
        #heads
        self.y1o = nn.Linear(128,race_shape)
        nn.init.xavier_normal_(self.y1o.weight)#
        self.y2o = nn.Linear(128,class_shape)
        nn.init.xavier_normal_(self.y2o.weight)
        
    def forward(self, x):
       
        x1 = self.resnet_model(x)
        #x1 =  F.relu(self.x1(x1))
        #x1 =  F.relu(self.x2(x1))
        
        x1 =  F.relu(self.x1(x1))
        x1 =  F.relu(self.x1a(x1))
        x1 =  F.relu(self.x2(x1))
        x1 =  F.relu(self.x2a(x1))
        
        x1 =  F.relu(self.x3(x1))
        x1 =  F.relu(self.x3a(x1))
        
        
        #x = F.relu(self.x2(x))
        #x1 = F.relu(self.x3(x))
        
        # heads
        y1o = F.softmax(self.y1o(x1),dim=1)
        y2o = F.softmax(self.y2o(x1),dim=1)
        #y2o = torch.sigmoid(self.y2o(x1)) #should be sigmoid


        return y1o, y2o
model_1 = multi_output_model(model_ft)
model_1 = model_1.to(device)
model_1.load_state_dict(torch.load('./models/masked_resnet34.pth'))

print(model_1)
print(model_1.parameters())    
criterion = [nn.CrossEntropyLoss(),nn.CrossEntropyLoss()]

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=F

)
multi_output_model(
  (resnet_model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace)
        (downsample): Sequential(
  

)
<generator object Module.parameters at 0x0000024585B3E840>


In [19]:
it = iter(dataloaders_style_dict['val'])
gender_input ,gender = next(it)
x1,x2 = model_1(gender_input.to(device) )
x1,x2


(tensor([[1.1028e-02, 9.6551e-01, 2.3460e-02],
         [7.6354e-04, 9.9167e-01, 7.5649e-03],
         [7.2412e-01, 2.3155e-02, 2.5273e-01],
         [9.5255e-02, 5.8714e-01, 3.1760e-01],
         [2.4761e-03, 9.4278e-01, 5.4747e-02],
         [5.7927e-01, 2.2151e-01, 1.9923e-01],
         [1.6586e-01, 4.0949e-01, 4.2465e-01],
         [9.9525e-02, 4.6703e-02, 8.5377e-01]], device='cuda:0',
        grad_fn=<SoftmaxBackward>), tensor([[0.0174, 0.0246, 0.9579],
         [0.0042, 0.1050, 0.8908],
         [0.9657, 0.0195, 0.0148],
         [0.3038, 0.2287, 0.4675],
         [0.0053, 0.9885, 0.0062],
         [0.7765, 0.1442, 0.0793],
         [0.4382, 0.5468, 0.0150],
         [0.2207, 0.7228, 0.0566]], device='cuda:0', grad_fn=<SoftmaxBackward>))

In [20]:
from torch.autograd import Variable
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()
    print('starting')
    best_model_wts = copy.deepcopy(model.state_dict())
    best_loss = 100

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                scheduler.step()
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_loss0= 0.0
            running_loss1= 0.0
            
            #color_corrects= []
            #total_colors = []
            gender_corrects = 0.0
            color_corrects= 0.0
            
            num_class_preds = 0
            num_gender_preds = 0 
            # Iterate over data.
            gender_iterator = iter(dataloaders_dict[phase])

            style_iterator = iter(dataloaders_style_dict[phase])
            
            #number_of_batches = len(dataloaders_style_dict[phase]) + len(dataloaders_dict[phase])
            for i in range(len(dataloaders_style_dict[phase])):
                #dataloaders_dict , dataloaders_style_dict
                
                ### gender
                inputs,labels_ = next(gender_iterator)
                
                inputs = inputs.to(device) 
                labels_ = labels_.to(device)
                
                outputs = model(inputs)
                preds_ = outputs[0]
                
                loss_gender = criterion[0](preds_, torch.max(labels_.float(), 1)[1]).to(device)
                running_loss0 += loss_gender.item() * inputs.size(0)
                gender_corrects += torch.sum(torch.max(preds_, 1)[1] == torch.max(labels_, 1)[1])
                num_gender_preds += inputs.size(0)
                #loss = loss_gender
                #running_loss += loss.item() * inputs.size(0)
                
                #optimizer.zero_grad()
                
                #if phase == 'train':
                        
                #        loss.backward(retain_graph=True)
                #        optimizer.step()
                        
                #####################################################################################
                ### style
                inputs,labels_ = next(style_iterator)
                inputs = inputs.to(device) 
                labels_ = labels_.to(device)
                
                outputs = model(inputs)
                preds_ = outputs[1]

                loss_style = criterion[1](preds_, torch.max(labels_.float(), 1)[1]).to(device)
                running_loss1 += loss_style.item() * inputs.size(0)
                color_corrects += torch.sum(torch.max(preds_, 1)[1] == torch.max(labels_, 1)[1])
                num_class_preds += inputs.size(0)
                #loss = loss_style
                
                
                optimizer.zero_grad()
                
                loss = loss_style + loss_gender
                running_loss += loss.item() * inputs.size(0)*2
                if phase == 'train':
                        
                    loss.backward()
                    optimizer.step()
            
                
            epoch_loss = running_loss / (num_class_preds +num_gender_preds)
            epoch_loss0 = running_loss0 / (num_gender_preds)
            epoch_loss1 = running_loss1 / (num_class_preds)

            
            color_acc = color_corrects.double()/num_class_preds
            gender_corrects = gender_corrects.double() / num_gender_preds
            
            print('{} total loss: {:.4f} dnd_race loss: {:.4f} dnd_class loss: {:.4f}'.format(phase,epoch_loss,epoch_loss0,
                                                                                                          epoch_loss1))
            print('{} race_acc: {:.4f} '
                  'class_acc: {:.4f}'.format(
                phase, gender_corrects,color_acc))

            if phase == 'val' and epoch_loss < best_loss:
                print('saving with loss of {}'.format(epoch_loss),
                      'improved over previous {}'.format(best_loss))
                best_loss = epoch_loss
                best_model_wts = copy.deepcopy(model.state_dict())
                torch.save(best_model_wts, './models/masked_resnet34.pth')


        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(float(best_loss)))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [21]:
lrmain = .000001
lrlast = .0001
optim1 = optim.Adam(
    [
        {"params":model_1.resnet_model.parameters(),"lr": lrmain},
        {"params":model_1.x1.parameters(), "lr": lrlast},
        {"params":model_1.x1a.parameters(), "lr": lrlast},
        {"params":model_1.x2.parameters(), "lr": lrlast},
        {"params":model_1.x2a.parameters(), "lr": lrlast},
       {"params":model_1.x3.parameters(), "lr": lrlast},
       {"params":model_1.x3a.parameters(), "lr": lrlast},
       {"params":model_1.y1o.parameters(), "lr": lrlast},
         {"params":model_1.y2o.parameters(), "lr": lrlast},
       
       
   ])

#optim1 = optim.Adam(model_1.parameters(), lr=0.01)#,momentum=.9)
# Observe that all parameters are being optimized
optimizer_ft = optim1

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=6, gamma=0.1)

In [22]:
model_ft1 = train_model(model_1, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=30)

starting
Epoch 0/29
----------


D:\Users\micha\Anaconda3\envs\tensorflow_env\lib\site-packages\PIL\TiffImagePlugin.py:785: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))
D:\Users\micha\Anaconda3\envs\tensorflow_env\lib\site-packages\PIL\Image.py:965: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  ' expressed in bytes should be converted ' +


train total loss: 1.7396 dnd_race loss: 0.8834 dnd_class loss: 0.8562
train race_acc: 0.6585 class_acc: 0.6860
val total loss: 1.8055 dnd_race loss: 0.9258 dnd_class loss: 0.8797
val race_acc: 0.6125 class_acc: 0.6625
saving with loss of 1.8055377364158631 improved over previous 100

Epoch 1/29
----------
train total loss: 1.7609 dnd_race loss: 0.9038 dnd_class loss: 0.8571
train race_acc: 0.6494 class_acc: 0.6707
val total loss: 1.7175 dnd_race loss: 0.8677 dnd_class loss: 0.8498
val race_acc: 0.6500 class_acc: 0.6750
saving with loss of 1.7175435781478883 improved over previous 1.8055377364158631

Epoch 2/29
----------
train total loss: 1.7490 dnd_race loss: 0.9037 dnd_class loss: 0.8453
train race_acc: 0.6311 class_acc: 0.7104
val total loss: 1.9169 dnd_race loss: 0.9560 dnd_class loss: 0.9609
val race_acc: 0.5375 class_acc: 0.5875

Epoch 3/29
----------
train total loss: 1.7063 dnd_race loss: 0.8624 dnd_class loss: 0.8439
train race_acc: 0.6951 class_acc: 0.6982
val total loss: 1.7

KeyboardInterrupt: 

In [ ]:


gender = ['gender_Female',
       'gender_Male']

for i in gender:
    print(y_train[i].value_counts())

In [ ]:
237/(237+89)

# Eval
 

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
dd = .2
model_ft = models.resnet50(pretrained=True)
for param in model_ft.parameters():
    param.requires_grad = False
#print(model_ft)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, 512)

model1 = multi_output_model(model_ft,dd)
#print(model1)
model1 = model1.to(device)
model1.load_state_dict(torch.load('models/res502.pth'))
model1.eval()


In [ ]:
from torch.autograd import Variable

imsize = 224
loader = transforms.Compose([transforms.Resize((imsize,imsize)),
                             transforms.ToTensor(),
                             transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

def image_loader(image_name):
    """load image, returns cuda tensor"""
    image = Image.open('test/'+image_name)
    image = image.convert('RGB')
    
    image = loader(image).float()
    print(image.shape)
    image = Variable(image, requires_grad=False)
    image = image.unsqueeze(0)  #this is for VGG, may not be needed for ResNet
    return image.cuda()  #assumes that you're using GPU

#saber4.png

image = image_loader('jalter3.jpg')
y_pred = model1(image)


In [ ]:
len(y_pred)

In [ ]:
values, indices = torch.max(y_pred[1], -1)
print(y_pred[1],values, indices)

torch.topk(y_pred[1],1)[1]

In [ ]:
   
gender = ['Female',
       'Male']
region = ['Asia', 'Egypt', 'Europe',
       'Middle East']
fighting_style = ['magic', 'melee',
       'ranged']
alignment = ['CE', 'CG', 'CN',
       'LE', 'LG', 'LN', 'NE',
       'NG', 'TN']
colors = ['white', 'red',
       'green', 'black', 'blue', 'purple', 'gold', 'silver']

def extract_label(label_list, pred_array,top_n=1):
    pred_max = torch.topk(pred_array,top_n)[1]
    print(pred_max)
    out_list = []
    for i in pred_max[0]:
        out_list.append(label_list[i])
    return out_list

print(extract_label(gender,y_pred[0]))
print(extract_label(region,y_pred[1]))
print(extract_label(fighting_style,y_pred[2]))
print(extract_label(alignment,y_pred[3]))
print(extract_label(colors,y_pred[4],3))


In [ ]:
from os import listdir
from matplotlib.pyplot import imshow

%matplotlib inline
#pil_im = Image.open('data/empire.jpg', 'r')

test = listdir('test/')
    
for i in test:
    

    image = image_loader(i)
    y_pred = model1(image)
    print(i)
    print('')
    print('image: ', i )
    print(extract_label(gender,y_pred[0]))
    print(extract_label(region,y_pred[1]))
    print(extract_label(fighting_style,y_pred[2]))
    print(extract_label(alignment,y_pred[3]))
    print(extract_label(colors,y_pred[4],3))
    pil_im = Image.open('test/'+i)
    
    plt.imshow(np.asarray(pil_im))
    plt.xticks([]), plt.yticks([])  # to hide tick values on X and Y axis
    plt.show()
    
    